In [ ]:
# 标题: 依赖与路径
import os, pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

A_DIR = Path('/workspace/outputs/A-256')
B_DIR = Path('/workspace/outputs/B-256')

plt.rcParams['figure.figsize']=(6,4)

print('A_DIR exists:', A_DIR.exists())
print('B_DIR exists:', B_DIR.exists())


In [ ]:
# 标题: 汇总统计（epoch_init/epoch4）

def load_pseudo(base: Path):
    out={}
    for sub in ['epoch_init','epoch4']:
        p=base/ sub / 'ins_pseudo_label_train.p'
        if not p.exists():
            continue
        d=pickle.load(open(p,'rb'))
        tumor=[]; normal=[]; y=[]; s=[]
        for bag,m in d.items():
            for k,v in m.items():
                if 'tumor' in bag:
                    tumor.append(v); y.append(1); s.append(v)
                else:
                    normal.append(v); y.append(0); s.append(v)
        tumor=np.array(tumor); normal=np.array(normal)
        stats={
            'n_total': int(len(s)),
            'n_tumor': int(len(tumor)),
            'n_normal': int(len(normal)),
            'mean_tumor': float(np.mean(tumor)) if len(tumor)>0 else np.nan,
            'mean_normal': float(np.mean(normal)) if len(normal)>0 else np.nan,
            'p>0.5_tumor': float(np.mean(tumor>0.5)) if len(tumor)>0 else np.nan,
            'p>0.5_normal': float(np.mean(normal>0.5)) if len(normal)>0 else np.nan,
        }
        try:
            auc=roc_auc_score(y,s)
        except Exception:
            auc=np.nan
        stats['instance_auc']=auc
        out[sub]=stats
    return out

stats_A=load_pseudo(A_DIR)
stats_B=load_pseudo(B_DIR)

pd.DataFrame({'A-256':stats_A, 'B-256':stats_B}).T


In [ ]:
# 标题: 直方图对比（epoch4）
import matplotlib.pyplot as plt

def plot_hist(base: Path, title: str):
    p=base/'epoch4'/'ins_pseudo_label_train.p'
    if not p.exists():
        print('missing:', p)
        return
    d=pickle.load(open(p,'rb'))
    tumor=[]; normal=[]
    for bag,m in d.items():
        for k,v in m.items():
            (tumor if 'tumor' in bag else normal).append(v)
    plt.figure()
    plt.hist(tumor, bins=30, alpha=0.6, label='tumor', color='red')
    plt.hist(normal, bins=30, alpha=0.6, label='normal', color='blue')
    plt.axvline(0.5, ls='--', color='k', alpha=0.5)
    plt.title(title+' epoch4 pseudo scores')
    plt.legend(); plt.show()

plot_hist(A_DIR, 'A-256')
plot_hist(B_DIR, 'B-256')


In [ ]:
# 标题: 指标条形图（epoch_init vs epoch4）
import pandas as pd

def to_df(tag, stats):
    rows=[]
    for ep, m in stats.items():
        rows.append({'exp':tag,'epoch':ep,'auc':m.get('instance_auc',float('nan')),
                     'p>0.5_tumor':m.get('p>0.5_tumor',float('nan')),
                     'p>0.5_normal':m.get('p>0.5_normal',float('nan'))})
    return pd.DataFrame(rows)

df=pd.concat([to_df('A-256', stats_A), to_df('B-256', stats_B)], ignore_index=True)

for metric in ['auc','p>0.5_tumor','p>0.5_normal']:
    pivot=df.pivot(index='exp', columns='epoch', values=metric)
    cols=[c for c in ['epoch_init','epoch4'] if c in pivot.columns]
    if cols:
        pivot[cols].plot(kind='bar', title=metric)
        plt.axhline(0.5 if metric=='auc' else 0.5, ls='--', color='k', alpha=0.3)
        plt.show()
